In [34]:
import numpy as np
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import cvxpy as cp
from datetime import date, timedelta
from scipy.optimize import minimize
import warnings

from scipy.stats import skew, kurtosis

warnings.filterwarnings('ignore')

#### Comment this section in if your are having rate limit issues.
#### Otherwise, run the code as normal

In [35]:

# data = pd.read_csv('data.csv')
# data['Date'] = pd.to_datetime(data['Date'])
# Convert the 'Date' column to datetime objects after loading the data
etfs = ['DBA',	'EPP',	'EWJ',	'FEZ',	'FXE',	'GLD',	'ILF',	'QQQ',	'SHV',	'SPY',	'USO','XBI'	]

In [ ]:
etf_data = yf.download(etfs, start='2007-03-01', end='2024-12-31')['Close']


etf_returns = etf_data.pct_change().dropna()

In [ ]:
ff_factors = pd.read_csv('F-F_Research_Data_Factors_daily.CSV', skiprows=3, index_col=0)
ff_factors.dropna(inplace=True)
ff_factors.tail()
ff_factors = ff_factors.loc['2007-03-01':]
ff_factors.head()

In [45]:

etf_returns = etf_data.pct_change().dropna()

# Convert ff_factors index to datetime if it's not already
ff_factors.index = pd.to_datetime(ff_factors.index, format='%Y%m%d')

# Align the data
etf_returns = etf_returns.reindex(ff_factors.index).dropna()
ff_factors = ff_factors.reindex(etf_returns.index).dropna()

# Create merged data
data = pd.concat([etf_returns, ff_factors], axis=1)

In [ ]:
data.reset_index(inplace=True)
data.rename(columns={'index': 'Date'}, inplace=True)

#### Optimization routines 

In [48]:
def strategy_I(expected_returns, cov_matrix, betas, beta_constraints, lambd):
    """
    Optimizes portfolio weights using the best aspects of previous implementations.

    Parameters:
    - expected_returns: Expected returns for each asset (1D array)
    - cov_matrix: Covariance matrix of returns (2D array)
    - factor_loadings: DataFrame of factor exposures (rows: assets, columns: factors)
    - beta_col: The column name in factor_loadings to use for beta constraint (e.g., 'Mkt-RF')
    - beta_constraints: Tuple of (min_beta, max_beta)
    - lambd: Risk aversion parameter

    Returns:
    - Optimal portfolio weights (1D numpy array)
    """
    n = len(expected_returns)
    w = cp.Variable(n)
    # Ensure covariance matrix is symmetric for numerical stability
    cov_matrix = (cov_matrix + cov_matrix.T) / 2
    # Portfolio return
    portfolio_return = expected_returns @ w
    # Portfolio variance
    portfolio_risk = cp.quad_form(w, cov_matrix)
    # Portfolio beta (generalized to any factor column)
    portfolio_beta = betas @ w
    # Constraints

    constraints = [
        cp.sum(w) == 1,
        portfolio_beta >= beta_constraints[0],
        portfolio_beta <= beta_constraints[1],
        w >= -2,
        w <= 2
    ]
    # Objective: maximize risk-adjusted return (using standard deviation)
    objective = cp.Maximize(portfolio_return - lambd * portfolio_risk)
    prob = cp.Problem(objective, constraints)
    prob.solve()
    return w.value

In [49]:
def strategy_II(expected_returns, returns_data, betas, beta_constraints, lambd, benchmark_returns, covar):
    """
    Optimize portfolio weights using Strategy II approach

    Parameters:
    - expected_returns: Expected returns for each asset (numpy array)
    - returns_data: Historical returns data (DataFrame)
    - betas: calculated betas of each ETF (numpy array)
    - beta_constraints: Tuple of (min_beta, max_beta)
    - lambd: Risk aversion parameter
    - benchmark_returns: Returns of benchmark portfolio (Series)
    - covar: covariance matrix (DataFrame)
    """
    n = len(expected_returns)

    # Helper functions (moved inside)
    def TEV(weights):
        """Tracking error volatility"""
        portfolio_returns = returns_data @ weights
        tracking_error = portfolio_returns - benchmark_returns
        return np.sqrt(np.var(tracking_error))

    def portRisk(weights):
        """Portfolio risk"""
        return weights.T @ covar @ weights

    # Find SPY index if it exists
    spy_idx = None
    if 'SPY' in returns_data.columns:
        spy_idx = returns_data.columns.get_loc('SPY')

    def objective(weights):
        # Calculate portfolio return relative to benchmark
        if spy_idx is not None:
            port_return = np.dot(weights, expected_returns) - expected_returns[spy_idx]
        else:
            port_return = np.dot(weights, expected_returns) - np.mean(benchmark_returns)

        return -(port_return/TEV(weights) - lambd * np.sqrt(portRisk(weights)))

    # Constraints
    constraints = [
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # Full investment
        {'type': 'ineq', 'fun': lambda w: beta_constraints[1] - np.dot(w, betas)},  # Max beta
        {'type': 'ineq', 'fun': lambda w: np.dot(w, betas) - beta_constraints[0]},  # Min beta
    ]

    # Optimization setup
    result = minimize(
        objective,
        x0=np.ones(n)/n,  # Equal weights initial guess
        bounds=[(-2, 2)]*n,
        constraints=constraints,
        method='SLSQP',
        options={'maxiter': 1000}
    )

    return result.x if result.success else np.ones(n)/n  # Fallback to equal weights if fails

#### Helper Methods

In [50]:

def getRetCovEst(retLag, covLag, date):
    etf_returns = data

    etf_returns['Date'] = pd.to_datetime(etf_returns['Date'])

    #change to grab # of obs instead of cal days
    covRets = etf_returns[(etf_returns['Date'] <= date)].sort_values(by='Date',ascending=False).head(covLag)

    cov = covRets[etfs].cov()
    factorRets = etf_returns[etf_returns['Date'] <= date].sort_values(by='Date',ascending=False).head(retLag)
    market_risk_premiums = pd.DataFrame(index=etf_returns.columns, columns=['Market Risk Premium'])
    return_est = pd.DataFrame(index=etf_returns.columns, columns=['ExpectedReturn'])


    # Run regression for each ETF to get factor exposures
    X = sm.add_constant(factorRets[['Mkt-RF','SMB','HML']],has_constant='add')

    for etf in etfs:
        y = factorRets[[etf]]
        # Run regression
        model = sm.OLS(y, X).fit()
        # Calculate market risk premium and estimate returns
        beta1 = model.params['Mkt-RF']
        avg_market_return = factorRets['Mkt-RF'].mean()
        market_risk_premium = beta1 * avg_market_return
        beta2 = model.params['SMB']
        avg_SMB = factorRets['SMB'].mean()
        beta3 = model.params['HML']
        avg_HML = factorRets['HML'].mean()
        market_risk_premiums.loc[etf, 'Market Risk Premium'] = market_risk_premium
        expected_ret = factorRets['RF'].mean() + market_risk_premium + beta2 * avg_SMB + beta3 * avg_HML
        return_est.loc[etf,'ExpectedReturn'] = expected_ret

    #calculate betas - SPY will always have a beta of 1
    betas = pd.DataFrame(index=etfs, columns=['Beta'])
    for etf in etfs:
        if etf == "SPY":
            betas.loc[etf,'Beta'] = 1
        else:
            tickRet = factorRets[['Date',etf]]
            spyrets = factorRets[['Date','SPY']]
            mergedRet = pd.merge(tickRet, spyrets,how='left',on='Date')
            tcov = mergedRet[[etf,'SPY']].cov()
            beta = tcov.iloc[0][1]/tcov.iloc[1][1]
            betas.loc[etf, 'Beta']= beta

    return cov, betas, return_est.dropna(), factorRets

In [51]:

def getRebalDates(ret_dates, min, max):
    rebalDates = []
    mondays = []
    current_date = min
    #get all mondays between our min and max date
    while current_date.weekday() != 0:  # Monday is 0
        current_date += timedelta(days=1)
    while current_date <= max:
        mondays.append(current_date)
        current_date += timedelta(days=7)
    #get each monday in our analysis period
    for d in mondays:
        #if monday is not in our return days (proxy for us trading days), add the tuesday
        #we can assume that if we're missing monday it's because monday is a market holiday
        #if monday is a market holiday, Tuesday MUST bet a trading day - markets cant be closed for 4 days in a row
        if d in ret_dates:
            rebalDates.append(d)
        else:
            rebalDates.append(d + timedelta(days=1))
    return rebalDates



In [52]:

def calculate_performance_metrics(returns):
    """Calculate key performance metrics from return series"""
    metrics = {}

    # Cumulative returns
    metrics['Cumulative Return'] = (1 + returns).prod() - 1

    # Daily returns
    metrics['Daily Mean (Arith)'] = returns.mean()
    metrics['Daily Mean (Geom)'] = ((1 + returns).prod()) ** (1/len(returns)) - 1
    metrics['Daily Min Return'] = returns.min()

    # Risk metrics
    metrics['Volatility'] = returns.std() * np.sqrt(252)
    metrics['Sharpe Ratio'] = metrics['Daily Mean (Geom)'] * np.sqrt(252) / metrics['Volatility']

    # Drawdown
    cum_returns = (1 + returns).cumprod()
    peak = cum_returns.expanding().max()
    drawdown = (cum_returns - peak) / peak
    metrics['Max Drawdown'] = drawdown.min()

    # Higher moments
    metrics['Skewness'] = returns.skew()
    metrics['Kurtosis'] = returns.kurtosis()

    # Tail risk
    metrics['VaR (95%)'] = returns.quantile(0.05)
    metrics['CVaR (95%)'] = returns[returns <= returns.quantile(0.05)].mean()

    return metrics

#### Code to run full backtest

In [53]:
def runBacktest(startdt, enddt, ret_est_per, cov_est_per, lambda_val):
    """
    Run backtest for both strategies between start and end dates

    Parameters:
    - startdt: Start date (YYYY-MM-DD)
    - enddt: End date (YYYY-MM-DD)
    - ret_est_per: Returns estimation period (days)
    - cov_est_per: Covariance estimation period (days)
    - lambda_val: Risk aversion parameter

    Returns:
    - Dictionary with weights, returns, and portfolio values
    """
    # Convert and validate dates
    startdt = pd.to_datetime(startdt)
    enddt = pd.to_datetime(enddt)

    # Filter data for backtest period
    period_data = data[(data['Date'] >= startdt) & (data['Date'] <= enddt)].copy()

    # Initialize results DataFrames with proper datetime index
    results = {
        'weights_I': pd.DataFrame(index=period_data['Date'].unique(), columns=etfs),
        'weights_II': pd.DataFrame(index=period_data['Date'].unique(), columns=etfs),
        'daily_returns': pd.DataFrame(index=period_data['Date'], columns=['Strategy_I', 'Strategy_II', 'SPY']),
        'portfolio_values': pd.DataFrame(index=period_data['Date'], columns=['Strategy_I', 'Strategy_II', 'SPY'])
    }

    # Initialize portfolio values
    results['portfolio_values'].iloc[0] = 100

    # Get rebalancing dates (weekly)
    rebalDates = getRebalDates(period_data['Date'].values, startdt, enddt)

    for i, rebal_date in enumerate(rebalDates):
        try:
            # Get estimation data
            cov, betas, retest, inputRets = getRetCovEst(ret_est_per, cov_est_per, rebal_date)

            # Optimize portfolios
            weights_I = strategy_I(
                retest['ExpectedReturn'].values,
                cov,
                betas['Beta'].values,
                [-0.5, 0.5],
                lambda_val
            )

            weights_II = strategy_II(
                retest['ExpectedReturn'].values,
                inputRets[etfs],
                betas['Beta'].values,
                [-2, 2],
                lambda_val,
                inputRets['SPY'],
                cov
            )

            # Store weights
            results['weights_I'].loc[rebal_date] = weights_I
            results['weights_II'].loc[rebal_date] = weights_II

            # Calculate period returns
            next_rebal = rebalDates[i+1] if i < len(rebalDates)-1 else enddt
            period_mask = (period_data['Date'] >= rebal_date) & (period_data['Date'] <= next_rebal)
            period_returns = period_data.loc[period_mask]

            # Calculate daily returns and portfolio values            
            for j in range(len(period_returns)):
                date = period_returns.iloc[j]['Date']
                daily_ret = period_returns.iloc[j][etfs].values

                # Calculate strategy returns
                ret_I = np.dot(weights_I, daily_ret)
                ret_II = np.dot(weights_II, daily_ret)
                # Store returns
                results['daily_returns'].loc[date, 'Strategy_I'] = ret_I
                results['daily_returns'].loc[date, 'Strategy_II'] = ret_II
                results['daily_returns'].loc[date, 'SPY'] = period_returns.iloc[j]['SPY']

                # Update portfolio values
                if j == 0 and i == 0:
                    prev_value_I = 100
                    prev_value_II = 100
                    prev_value_spy = 100
                else:
                    if j != 0:
                        prev_date = period_returns.iloc[j-1]['Date']
                    else:
                        prev_date = period_returns.iloc[0]['Date']
                    prev_value_I = results['portfolio_values'].loc[prev_date, 'Strategy_I']
                    prev_value_II = results['portfolio_values'].loc[prev_date, 'Strategy_II']
                    prev_value_spy = results['portfolio_values'].loc[prev_date, 'SPY']

                results['portfolio_values'].loc[date, 'Strategy_I'] = prev_value_I * (1 + ret_I)
                results['portfolio_values'].loc[date, 'Strategy_II'] = prev_value_II * (1 + ret_II)
                results['portfolio_values'].loc[date, 'SPY'] = prev_value_spy * (1 + period_returns.iloc[j]['SPY'])

        except Exception as e:
            print(f"Error processing {rebal_date}: {str(e)}")
            continue

    # Clean up results
    results['weights_I'] = results['weights_I'].dropna(how='all')
    results['weights_II'] = results['weights_II'].dropna(how='all')
    results['daily_returns'] = results['daily_returns'].dropna()
    results['portfolio_values'] = results['portfolio_values'].dropna()

    return results

In [ ]:
#test backtest method
t = runBacktest('2008-09-01', '2008-9-20',40,60,1)
t

### Full implementation - Run each regime, for each estimation period and each Lambda value 
#### Note: this takes ~2 hours to run, please use the "sample" method below to verify functionality 
#### the full period and bull market runs take particularly long - the 9 variations for the pre-subprie takes about 2 minutes

In [ ]:
# Define sub-periods
periods = {
    'Subprime Crisis (Before)': ('2007-03-01', '2008-08-31'),
    'Subprime Crisis (During)': ('2008-09-01', '2009-03-31'),
    '2010s Bull Market': ('2009-04-01', '2019-12-31'),
    'COVID (During)': ('2020-01-01', '2021-12-31'),
    'COVID (After)': ('2022-01-01', '2024-12-31'), # Assuming end of 2024 as per requirement
    'Whole Period': ('2007-03-01', '2024-12-31')
}

# Define lambda values
lambdas = [0.1, 0.5, 1]

# Define estimation periods
estimation_periods = [(40, 60), (90, 60),(180,60)] # (ret_est_per, cov_est_per) representing S40/60 and S90/60

# Create a table to store results
performance_table = pd.DataFrame()
ret_dict = {}
pnl_dict = {}
keys = []

for p in periods:
    period_name = p
    start_date, end_date = periods[period_name]
    for l in lambdas:
        lambda_val = l # lambda = 0.1
        for estp in estimation_periods:
            ret_per, cov_per = estp # S40/60
            
            key = p + "|" + str(l) +"|"+ str(ret_per)
            keys.append(key)

            print(f"Running backtest for: {period_name}, Estimation: {ret_per}/{cov_per}, Lambda: {lambda_val}")

            try:
                backtest_results = runBacktest(start_date, end_date, ret_per, cov_per, lambda_val)

                # Calculate performance metrics
                metrics_I = calculate_performance_metrics(backtest_results['daily_returns']['Strategy_I'])
                metrics_II = calculate_performance_metrics(backtest_results['daily_returns']['Strategy_II'])
                metrics_SPY = calculate_performance_metrics(backtest_results['daily_returns']['SPY'])
                ret_dict[key]=backtest_results['daily_returns']
                pnl_dict[key]=backtest_results['portfolio_values']
                # Add metrics to the table
                temp_df = pd.DataFrame({
                    'Period': [period_name, period_name, period_name],
                    'Estimation': [f'S{ret_per}/{cov_per}', f'S{ret_per}/{cov_per}', f'S{ret_per}/{cov_per}'],
                    'Lambda': [lambda_val, lambda_val, lambda_val],
                    'Strategy': ['Strategy I', 'Strategy II', 'SPY'],
                    'Cumulative Return': [metrics_I['Cumulative Return'], metrics_II['Cumulative Return'], metrics_SPY['Cumulative Return']],
                    'Annualized Volatility': [metrics_I['Volatility'], metrics_II['Volatility'], metrics_SPY['Volatility']],
                    'Sharpe Ratio': [metrics_I['Sharpe Ratio'], metrics_II['Sharpe Ratio'], metrics_SPY['Sharpe Ratio']],
                    'Max Drawdown': [metrics_I['Max Drawdown'], metrics_II['Max Drawdown'], metrics_SPY['Max Drawdown']],
                    'Skewness': [metrics_I['Skewness'], metrics_II['Skewness'], metrics_SPY['Skewness']],
                    'Kurtosis': [metrics_I['Kurtosis'], metrics_II['Kurtosis'], metrics_SPY['Kurtosis']],
                    'VaR (95%)': [metrics_I['VaR (95%)'], metrics_II['VaR (95%)'], metrics_SPY['VaR (95%)']],
                    'CVaR (95%)': [metrics_I['CVaR (95%)'], metrics_II['CVaR (95%)'], metrics_SPY['CVaR (95%)']]
                })

                performance_table = pd.concat([performance_table, temp_df], ignore_index=True)

            except Exception as e:
                print(f"Backtest failed for {period_name}: {e}")

# Display the resulting table for the first run
print("\nPerformance Table (First Period, First Estimation, First Lambda):")
print(performance_table.to_markdown(index=False))


In [82]:
#output results to csv for easier viz, analysis 
performance_table.to_csv("All Results.csv")

In [ ]:
import matplotlib.pyplot as plt
for p in keys:
    data = pnl_dict[p]
    plt.figure(figsize=(7, 4))
    plt.plot(data['SPY'],label='SPY')
    plt.plot(data['Strategy_I'],label='Strat 1')
    plt.plot(data['Strategy_II'],label='Strat 2')
    plt.title('pnl ' + p)
    plt.legend()
    plt.show()


In [ ]:

for p in keys:
    data = pnl_dict[p]
    plt.figure(figsize=(7, 4))
    plt.hist(data['SPY'], bins=20, alpha=0.7, label='SPY')
    plt.hist(data['Strategy_I'], bins=20, alpha=0.7, label='Strat 1')
    plt.hist(data['Strategy_II'], bins=20, alpha=0.7, label='Strat 2')
    plt.title('Daily Ret Dis ' + p)
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(True)
    plt.show()


#### Sample method for logic verification

In [ ]:
# Define sub-periods
periods = {
    'Subprime Crisis (Before)': ('2007-03-01', '2008-08-31')
    # 'Subprime Crisis (During)': ('2008-09-01', '2009-03-31'),
    # '2010s Bull Market': ('2009-04-01', '2019-12-31'),
    # 'COVID (During)': ('2020-01-01', '2021-12-31'),
    # 'COVID (After)': ('2022-01-01', '2024-12-31'), # Assuming end of 2024 as per requirement
    # 'Whole Period': ('2007-03-01', '2024-12-31')
}

# Define lambda values
lambdas = [0.1, 0.5, 1]

# Define estimation periods
estimation_periods = [(40, 60), (90, 60),(180,60)] # (ret_est_per, cov_est_per) representing S40/60 and S90/60

# Create a table to store results
performance_table_s = pd.DataFrame()
ret_dict_s = {}
pnl_dict_s = {}
keys_s = []

for p in periods:
    period_name = p
    start_date, end_date = periods[period_name]
    for l in lambdas:
        lambda_val = l # lambda = 0.1
        for estp in estimation_periods:
            ret_per, cov_per = estp # S40/60
            
            key = p + "|" + str(l) +"|"+ str(ret_per)
            keys_s.append(key)

            print(f"Running backtest for: {period_name}, Estimation: {ret_per}/{cov_per}, Lambda: {lambda_val}")

            try:
                backtest_results = runBacktest(start_date, end_date, ret_per, cov_per, lambda_val)

                # Calculate performance metrics
                metrics_I = calculate_performance_metrics(backtest_results['daily_returns']['Strategy_I'])
                metrics_II = calculate_performance_metrics(backtest_results['daily_returns']['Strategy_II'])
                metrics_SPY = calculate_performance_metrics(backtest_results['daily_returns']['SPY'])
                ret_dict_s[key]=backtest_results['daily_returns']
                pnl_dict_s[key]=backtest_results['portfolio_values']
                # Add metrics to the table
                temp_df = pd.DataFrame({
                    'Period': [period_name, period_name, period_name],
                    'Estimation': [f'S{ret_per}/{cov_per}', f'S{ret_per}/{cov_per}', f'S{ret_per}/{cov_per}'],
                    'Lambda': [lambda_val, lambda_val, lambda_val],
                    'Strategy': ['Strategy I', 'Strategy II', 'SPY'],
                    'Cumulative Return': [metrics_I['Cumulative Return'], metrics_II['Cumulative Return'], metrics_SPY['Cumulative Return']],
                    'Annualized Volatility': [metrics_I['Volatility'], metrics_II['Volatility'], metrics_SPY['Volatility']],
                    'Sharpe Ratio': [metrics_I['Sharpe Ratio'], metrics_II['Sharpe Ratio'], metrics_SPY['Sharpe Ratio']],
                    'Max Drawdown': [metrics_I['Max Drawdown'], metrics_II['Max Drawdown'], metrics_SPY['Max Drawdown']],
                    'Skewness': [metrics_I['Skewness'], metrics_II['Skewness'], metrics_SPY['Skewness']],
                    'Kurtosis': [metrics_I['Kurtosis'], metrics_II['Kurtosis'], metrics_SPY['Kurtosis']],
                    'VaR (95%)': [metrics_I['VaR (95%)'], metrics_II['VaR (95%)'], metrics_SPY['VaR (95%)']],
                    'CVaR (95%)': [metrics_I['CVaR (95%)'], metrics_II['CVaR (95%)'], metrics_SPY['CVaR (95%)']]
                })

                performance_table_s = pd.concat([performance_table_s, temp_df], ignore_index=True)

            except Exception as e:
                print(f"Backtest failed for {period_name}: {e}")

# Display the resulting table for the first run
print("\nPerformance Table (First Period, First Estimation, First Lambda):")
print(performance_table_s.to_markdown(index=False))
